In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
from numpy.random import default_rng

In [ ]:
df_amplitude=pd.read_csv('/content/gdrive/My Drive/Thesis/lab_csi/downlink/amp_lab_down.csv',header=None)
df_phase=pd.read_csv('/content/gdrive/My Drive/Thesis/lab_csi/downlink/phase_lab_down.csv',header=None)


In [ ]:
df_amplitude.index+=1
df_amplitude.columns+=1
df_phase.index+=1
df_phase.columns+=1

In [ ]:
df_amplitude.shape

(18000, 5520)

In [ ]:
arr=np.array(df_amplitude.T)
arrphase=np.array(df_phase.T)

In [ ]:
narr_phase=arrphase.reshape(5520,3,30,200)
narr_amp=arr.reshape(5520,3,30,200)

In [ ]:
arr_merged = np.append(narr_amp,narr_phase, axis=2)
ready_for_cnn=np.array(arr_merged)
ready_for_cnn=ready_for_cnn.transpose(0,3,2,1)

In [ ]:
ready_for_cnn.shape

(5520, 200, 60, 3)

CNN part


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
c=np.arange(1,277)
h=np.array([c]*20)
v=h.flatten()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test= train_test_split(ready_for_cnn,v,test_size=.2,random_state =30,stratify=v)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder=OneHotEncoder(handle_unknown='ignore',sparse= False)

y_train= encoder.fit_transform(Y_train.reshape([-1,1]))
y_test=encoder.transform(Y_test.reshape([-1,1]))

In [ ]:
cnn = models.Sequential([
    #1st Layer
    layers.Conv2D(filters=3, kernel_size=(3, 3),padding="same",strides=(1,1), input_shape=(200, 60, 3)),
    layers.BatchNormalization(momentum=0.9,epsilon=0.000001),
    layers.Activation('relu'),
    layers.AveragePooling2D(pool_size=(3,3),strides=(3,3),padding="same"),
    layers.Dropout(rate=.6),

    layers.Flatten(),
    #layers.Dense(276, activation='relu'),
    layers.Dense(276, activation='softmax')
])


cnn.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# array_for_training_loss=[]
# array_for_training_accuracy=[]
# array_for_testing_loss=[]
# array_for_testing_accuracy=[]

# for i in range(10):

#      history=cnn.fit(X_train, y_train, epochs=10,batch_size=256,validation_data=(X_test,y_test))
#      array_for_training_loss.append(history.history['loss'])
#      array_for_training_accuracy.append(history.history['accuracy'])
#      array_for_testing_loss.append(history.history['val_loss'])
#      array_for_testing_accuracy.append(history.history['val_accuracy'])


In [ ]:
history=cnn.fit(X_train, y_train, epochs=100,batch_size=256,validation_data=(X_test,y_test))

In [ ]:
# test_acc=np.mean(np.array(array_for_testing_accuracy),axis=0)
# train_acc=np.mean(np.array(array_for_training_accuracy),axis=0)
# train_loss=np.mean(np.array(array_for_training_loss),axis=0)
# test_loss=np.mean(np.array(array_for_testing_loss),axis=0)

In [ ]:
y_hat=cnn.predict(X_test)

y_hat2=1+y_hat.argmax(axis=1)

print((y_hat2==Y_test).sum())

In [ ]:
h=cnn.evaluate(X_test,y_test)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# plt.plot(train_loss)
# plt.plot(test_loss)
# plt.title('Model_loss')
# plt.ylabel('loss')

# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper right')
# plt.show()

In [ ]:
# plt.plot(train_acc)
# plt.plot(test_acc)
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='lower right')
# plt.show()

In [ ]:
history.history.keys()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model_loss')
plt.ylabel('loss')

plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')
plt.show()

**Transfer LearningPart**

In [ ]:
from tensorflow import keras
model=keras.models.load_model('/content/gdrive/My Drive/Thesis/fine_tuning/labuplink_saved.hdf5')

In [ ]:
model.trainable= True

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 200, 60, 3)        84        
                                                                 
 batch_normalization (BatchN  (None, 200, 60, 3)       12        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 200, 60, 3)        0         
                                                                 
 average_pooling2d (AverageP  (None, 67, 20, 3)        0         
 ooling2D)                                                       
                                                                 
 dropout (Dropout)           (None, 67, 20, 3)         0         
                                                                 
 flatten (Flatten)           (None, 4020)              0

In [ ]:
len(model.trainable_variables)

6

In [ ]:
fine_tune_at=6

In [ ]:
for layer in model.layers[:fine_tune_at]:
  layer.trainable= False

In [ ]:
base_learning_rate=.01

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 200, 60, 3)        84        
                                                                 
 batch_normalization (BatchN  (None, 200, 60, 3)       12        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 200, 60, 3)        0         
                                                                 
 average_pooling2d (AverageP  (None, 67, 20, 3)        0         
 ooling2D)                                                       
                                                                 
 dropout (Dropout)           (None, 67, 20, 3)         0         
                                                                 
 flatten (Flatten)           (None, 4020)              0

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history_tl=model.fit(X_train, y_train, epochs=200,batch_size=128,validation_data=(X_test,y_test))

In [ ]:
plt.plot(history_tl.history['loss'])
plt.plot(history_tl.history['val_loss'])
plt.title('Model_loss')
plt.ylabel('loss')

plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
plt.plot(history_tl.history['accuracy'])
plt.plot(history_tl.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')
plt.show()